## Load Fin-Fan failure detail

In [1]:
#Dependencies
import pandas as pd
from sqlalchemy import create_engine



Extract CSV into DataFrames

In [2]:
failure_csv = "resources/HVAC_failures_2021.csv"

Transform CSV DataFrame

In [3]:
failure_df = pd.read_csv(failure_csv)
failure_df.head()


,Event name,Start time,End time,Duration,Event template,Primary element,Primary element path,Running status reset post event start?,Unnamed: 8,Difference,Comments,Failure
0,HVAC Fan S311HHU205B Bearing Failure 2020-01-1...,16/01/2020 14:11,16/01/2020 14:20,0:08:54,HVAC Fan Failed Event Frame,S311HHU205B,\\ICHTHYS_PIAF\Predictive Analytics\HVAC Fan\F...,9/02/2020 15:41,0.0,577.3433419,False Alarm (speed drop for few mins),False
1,HVAC Fan S311HHU209B Bearing Failure 2020-03-0...,8/03/2020 20:11,8/03/2020 20:51,0:40:10,HVAC Fan Failed Event Frame,S311HHU209B,\\ICHTHYS_PIAF\Predictive Analytics\HVAC Fan\F...,26/03/2020 22:39,0.0,433.8001192,False Alarm (speed drop for few mins),False
2,HVAC Fan S311HHU209B Bearing Failure 2020-04-2...,24/04/2020 10:02,24/04/2020 10:22,0:19:45,HVAC Fan Failed Event Frame,S311HHU209B,\\ICHTHYS_PIAF\Predictive Analytics\HVAC Fan\F...,11/05/2020 9:17,0.0,406.9254994,False Alarm (speed drop for few mins),False
3,HVAC Fan S311HHU209B Bearing Failure 2020-11-0...,5/11/2020 6:35,5/11/2020 6:47,0:11:53,HVAC Fan Failed Event Frame,S311HHU209B,\\ICHTHYS_PIAF\Predictive Analytics\HVAC Fan\F...,31/01/2021 7:18,0.0,2088.51602,False Alarm (speed drop for few mins),False
4,HVAC Fan S311HK017A Bearing Failure 2021-02-06...,6/02/2021 20:31,7/02/2021 7:23,10:52:13,HVAC Fan Failed Event Frame,S311HK017A,\\ICHTHYS_PIAF\Predictive Analytics\HVAC Fan\F...,7/02/2021 7:23,0.0,0,TRUE Failure,True


In [4]:
# Select only the columns required
new_failure_df = failure_df[['Primary element','Start time', 'End time','Duration','Difference','Comments','Failure']].copy()
new_failure_df.head()



,Primary element,Start time,End time,Duration,Difference,Comments,Failure
0,S311HHU205B,16/01/2020 14:11,16/01/2020 14:20,0:08:54,577.3433419,False Alarm (speed drop for few mins),False
1,S311HHU209B,8/03/2020 20:11,8/03/2020 20:51,0:40:10,433.8001192,False Alarm (speed drop for few mins),False
2,S311HHU209B,24/04/2020 10:02,24/04/2020 10:22,0:19:45,406.9254994,False Alarm (speed drop for few mins),False
3,S311HHU209B,5/11/2020 6:35,5/11/2020 6:47,0:11:53,2088.51602,False Alarm (speed drop for few mins),False
4,S311HK017A,6/02/2021 20:31,7/02/2021 7:23,10:52:13,0,TRUE Failure,True


In [7]:
# Rename columns
new_failure_df = new_failure_df.rename(columns={'Primary element':'primary_element', 'Start time': 'start_time', 'End time':'end_time', 'Duration':'duration', 'Difference':'difference', 'Comments':'comments', 'Failure':'failure'})
new_failure_df



,primary_element,start_time,end_time,duration,difference,comments,failure
0,S311HHU205B,16/01/2020 14:11,16/01/2020 14:20,0:08:54,577.3433419,False Alarm (speed drop for few mins),False
1,S311HHU209B,8/03/2020 20:11,8/03/2020 20:51,0:40:10,433.8001192,False Alarm (speed drop for few mins),False
2,S311HHU209B,24/04/2020 10:02,24/04/2020 10:22,0:19:45,406.9254994,False Alarm (speed drop for few mins),False
3,S311HHU209B,5/11/2020 6:35,5/11/2020 6:47,0:11:53,2088.51602,False Alarm (speed drop for few mins),False
4,S311HK017A,6/02/2021 20:31,7/02/2021 7:23,10:52:13,0,TRUE Failure,True
5,S311HK020A,16/01/2020 14:20,16/01/2020 14:27,0:07:16,0.240321667,False Alarm (speed drop for few mins),False
6,S311HK049B,11/02/2021 17:08,11/02/2021 17:14,0:06:00,0,Possible. Refer Trend,True
7,S311HK049B,11/02/2021 18:04,11/02/2021 18:10,0:06:00,0,Possible. Refer Trend,True
8,S311HK049B,11/02/2021 21:01,11/02/2021 21:07,0:06:00,0,Possible. Refer Trend,True
9,S311HK049B,12/02/2021 2:46,12/02/2021 2:52,0:06:00,0,Possible. Refer Trend,True


Load

In [ ]:
# Connect to local database
rds_connection_string = "postgres:postgres@localhost:5432/music_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
# Create a music_db. Add the tables into postgres database as per schema.sql
engine.table_names()

In [ ]:
# Load pandas dataframe new_csv_df to database music_db, table spotify_csv
new_csv_df.to_sql(name='spotify_csv', con=engine, if_exists='append', index=False)

In [ ]:
# Load pandas dataframe id_timesig_df to database music_db, table spotify_api
id_timesig_df.to_sql(name='spotify_api', con=engine, if_exists='append', index=False)

# Alternatively, id_timesig_df pandas dataframe has been saved as a csv file. This csv file can be loaded into postgres database directly in table 'spotify_api'
# Relative csv link location: resources/id_timesig_df.csv
